In [45]:
#packages
import numpy as np
import pandas as pd
import os

# 1) STEM stats for state

In [65]:
def process_stem_data(file_path):
    df = pd.read_excel(file_path, sheet_name="State")
    df = df[['area_name', 'STEM_group', 'tot_emp', 'h_mean', 'a_mean']]

    # Calculate the total employment by state
    total_employment_by_state = df.groupby('area_name')['tot_emp'].sum().reset_index()

    # Calculate the total STEM employment by state
    stem_employment_by_state = df[df['STEM_group'] == 'STEM'].groupby('area_name')['tot_emp'].sum().reset_index()

    # Merge the two dataframes to calculate the percentage of STEM employees
    employment_merged = pd.merge(total_employment_by_state, stem_employment_by_state, on='area_name', suffixes=('_total', '_STEM'))

    # Calculate the percentage of STEM employees
    employment_merged['STEM_percentage'] = (employment_merged['tot_emp_STEM'] / employment_merged['tot_emp_total']) * 100

    # Calculate average state income
    df['tot_hr_earn'] = df['h_mean'] * df['tot_emp']
    df['tot_yr_earn'] = df['a_mean'] * df['tot_emp']

    sumup = df.groupby('area_name').agg({"tot_hr_earn": "sum", "tot_yr_earn": "sum", "tot_emp": "sum"})

    sumup['state_avg_hr_earn'] = sumup['tot_hr_earn'] / sumup['tot_emp']
    sumup['state_avg_yr_earn'] = sumup['tot_yr_earn'] / sumup['tot_emp']

    # Merge
    state = pd.merge(employment_merged, sumup, on='area_name', suffixes=('_total', '_STEM'))
    state = state[['area_name', 'tot_emp_STEM', 'STEM_percentage', 'state_avg_hr_earn', 'state_avg_yr_earn']]

    # New additions
    df_stat = df[df['STEM_group']=="STEM"][['area_name','h_mean','a_mean']]
    df_m = pd.merge(df_stat, state, on='area_name', suffixes=('_total', '_STEM'))

    df_m['avg_wagegap(hr)_STEM_vs_state'] = df_m['h_mean'] - df_m['state_avg_hr_earn']
    df_m['avg_wagegap(yr)_STEM_vs_state'] = df_m['a_mean'] - df_m['state_avg_yr_earn']

    df_m = df_m[['area_name','h_mean','avg_wagegap(hr)_STEM_vs_state','a_mean','avg_wagegap(yr)_STEM_vs_state','tot_emp_STEM','STEM_percentage']]

    return df_m

# List of years to process
years = [2019,2020,2021, 2022, 2023]

# Initialize an empty list to store DataFrames
dfs = []

# Loop through the years
for year in years:
    file_name = f"stem_{year}.xlsx"
    file_path = os.path.join("Data", file_name)
    
    if os.path.exists(file_path):
        df = process_stem_data(file_path)
        df['year'] = year  # Add a year column
        dfs.append(df)
    else:
        print(f"File not found: {file_path}")

# Combine all DataFrames
combined_df = pd.concat(dfs, ignore_index=True)

# Display the first few rows of the combined DataFrame
combined_df.head()

,area_name,h_mean,avg_wagegap(hr)_STEM_vs_state,a_mean,avg_wagegap(yr)_STEM_vs_state,tot_emp_STEM,STEM_percentage,year
0,Alabama,42.81,21.222078,89040,44144.188292,109640,5.553726,2019
1,Alaska,43.81,15.323030,91120,31862.917153,20280,6.395661,2019
2,Arizona,42.05,17.585005,87460,36572.323663,186700,6.512465,2019
3,Arkansas,36.49,15.977500,75890,33232.047691,48470,3.981370,2019
4,California,54.50,25.063509,113360,52133.207710,1312170,7.548843,2019


In [64]:
combined_df.to_csv("Data/stem_stat_by_state.csv")

In [74]:
df = pd.read_excel("Data/pce1023.xlsx",sheet_name="Table 4")

df = df[['Unnamed: 0','Total personal consumption expenditures']]

df = df.rename(columns=dict(zip(df.columns.values,['area_name','livingCost'])))

df.to_csv("Data/livingcost.csv")

# 2) Instutions stat for states